<a href="https://colab.research.google.com/github/shanaka-desoysa/notes/blob/main/docs/Misc/Alaska_RV_Parks_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommended RV Parks Dataset

https://www.greatalaskanholidays.com/trip-planning/recommended-campgrounds/


## Setup

In [1]:
!pip install polyline geocoder -q

     |████████████████████████████████| 98 kB 5.2 MB/s 


In [2]:
import pandas as pd

## Scrape Data

In [3]:
df_all = pd.read_html('https://www.greatalaskanholidays.com/trip-planning/recommended-campgrounds/')

In [4]:
len(df_all)

3

### DataFrames


1.   Privately Owned Campgrounds
2.   State of Alaska Campgrounds



In [36]:
df = df_all[0].copy()
df.head(5)

,Campground Name,Sites,Website,Business Address,Email
0,Alaskan Angler RV Resort,73,https://alaskabestrvpark.com/,"15640 Kingsley Rd, Ninilchik, AK 99639",bakersrvresort@yahoo.com
1,Anchorage Ship Creek RV Park,147,NaN,"150 N. Ingra St, Anchorage, AK 99501",dsmith@bestofalaskatravel.com
2,Bayside RV Park,117,https://www.baysidervcabin.com/,"230 E. Egan St, Valdez, AK 99686",Bayside1@cvinternet.net
3,Bear Creek RV Park,40,http://bearcreekrv.com/,"33508 Lincoln Ave, Seward, AK 99664",bearcreekrvpk@gci.net
4,Big Bear RV Park,59,http://www.bigbearrv.net,"2010 Church St, Palmer, AK 99645",bigbear@mtaonline.net


In [37]:
df.columns = ['CampgroundName', 'Sites', 'Website', 'BusinessAddress', 'Email']
df.head(5)

,CampgroundName,Sites,Website,BusinessAddress,Email
0,Alaskan Angler RV Resort,73,https://alaskabestrvpark.com/,"15640 Kingsley Rd, Ninilchik, AK 99639",bakersrvresort@yahoo.com
1,Anchorage Ship Creek RV Park,147,NaN,"150 N. Ingra St, Anchorage, AK 99501",dsmith@bestofalaskatravel.com
2,Bayside RV Park,117,https://www.baysidervcabin.com/,"230 E. Egan St, Valdez, AK 99686",Bayside1@cvinternet.net
3,Bear Creek RV Park,40,http://bearcreekrv.com/,"33508 Lincoln Ave, Seward, AK 99664",bearcreekrvpk@gci.net
4,Big Bear RV Park,59,http://www.bigbearrv.net,"2010 Church St, Palmer, AK 99645",bigbear@mtaonline.net


In [23]:
import logging
import requests
import json
import polyline
import folium
from folium.plugins import MeasureControl
import geocoder

from functools import lru_cache

logger = logging.getLogger(__name__)
DEBUG = True

@lru_cache(maxsize=None)
def geocode(location):
    return _geocode(location)

def _geocode(location):
    import geocoder
    # g = geocoder.osm(location)
    g = geocoder.arcgis(location)
    return g.latlng

@lru_cache(maxsize=None)
def get_route(olat, olng, dlat, dlng):
    response = _get_route(olat, olng, dlat, dlng)
    return response

def _get_route(olat, olng, dlat, dlng):
    url = f'http://router.project-osrm.org/route/v1/driving/{olng},{olat};{dlng},{dlat}?alternatives=false&steps=false'
    # logger.debug(url)
    response = None

    try:
        logger.debug(f'====== OSRM: {url}')
        response = requests.get(url, verify=False)
    except Exception as ex:
        raise

    # logger.debug(response.text)
    if response and response.text:
        response_dict = json.loads(response.text)
        #possible = pd.DataFrame([{'Distance': (route['distance'] / 1000) *  0.621371 , route['weight_name']: route['weight']} for route in response_dict['routes']])
        return response_dict
    else:
        return None

def get_routing_map(origin, destination, zoom=5):
    orig_latlng = geocode(origin)
    dest_latlng = geocode(destination)

    resp = get_route(orig_latlng[0], orig_latlng[1], dest_latlng[0], dest_latlng[1])

    decoded = polyline.decode(resp["routes"][0]['geometry'])
    distance = resp["routes"][0]['distance'] * 0.000621371
    duration = resp["routes"][0]['duration'] / 60

    map2 = folium.Map(location=(orig_latlng[0], orig_latlng[1]), zoom_start=zoom,
                                    control_scale=True)
    # map2.add_child(MeasureControl(
    #     primary_length_unit='miles',
    #     secondary_length_unit='meters',
    #     primary_area_unit='acres',
    #     secondary_area_unit='sqmeters')
    # )

    folium.PolyLine(locations=decoded, color="blue").add_to(map2)

    print(f"{duration} minutes")
    print(f"{distance} miles")

    return map2


## Geocode Private Campgrounds

In [15]:
geocode('15640 Kingsley Rd, Ninilchik, AK 99639	')

[60.044940006940124, -151.66528099759728]

In [38]:
applied_df = df.apply(lambda row: geocode(row.BusinessAddress), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df = df.rename(columns={0:'lat', 1: 'lng'})

In [40]:
# df

In [41]:
df[pd.isnull(df.lat)]

,CampgroundName,Sites,Website,BusinessAddress,Email,lat,lng
18,Grand View RV Park,28,http://grandviewrv.com,Milepost 109.7 Glenn Highway,info@grandviewrv.com,NaN,NaN
42,Tok RV Village,162,http://tokrv.net,Mile 1313.4 Alaska Highway,camp@tokrv.net,NaN,NaN


In [55]:
# 61.84590432308156, -147.62491071636222
df.loc[df.CampgroundName == 'Grand  View RV Park', ['lat', 'lng']] = [61.84590432308156, -147.62491071636222]

# 63.33500236865673, -142.9639913435805
df.loc[df.CampgroundName == 'Tok  RV Village', ['lat', 'lng']] = [63.33500236865673, -142.9639913435805]

In [57]:
df[pd.isnull(df.lat)]

,CampgroundName,Sites,Website,BusinessAddress,Email,lat,lng


In [58]:
anchorage = geocode('Anchorage, Alaska, USA')

In [59]:
anchorage

[61.21753000000007, -149.85824999999997]

In [60]:
map2 = folium.Map(location=(anchorage[0], anchorage[1]), zoom_start=5, control_scale=True)

# add marker one by one on the map
for idx, row in df.iterrows():
    if not pd.isnull(row['lat']) and not pd.isnull(row['lng']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            popup=row['CampgroundName'],
        ).add_to(map2)

### Map of Privately Owned Campgrounds

In [61]:
map2

## State of Alaska Campgrounds

In [6]:
df_1 = df_all[1]

In [12]:
df_1 = df_1[~(df_1['Campground'] == df_1['Winter Open'])].reset_index(drop=True)

In [18]:
df_1 = df_1.fillna('')
df_1.head(5)

,Campground,RV,Fee,# Sites,Picnic Sites,Toilets,Water,Trails,Fishing,Nearest Community,Location,Winter Open
0,Bird Creek,Y,$20,27,Y,H,Y,Y,Y,Girdwood,Mi. 101.2 Seward Hwy,
1,Bird Creek Overflow,Y,$20,20,Y,H,H,H,Y,Girdwood,,
2,Eagle River*,,$25,57,Y,H,H,N,Y,Eagle River,Mi. 12.6 Glenn Hwy,
3,Eklutna,,$15,58,H,H,H,Y,Y,Eklutna,Mi. 26.5 Glenn Hwy,
4,Big Delta SHP,Y,$20,25,Y,H,H,Y,N,Delta Junction,Mi. 274.5 Richardson Hwy,


In [21]:
df_1['Address'] = df_1['Campground'] + ', ' + df_1['Location'] + ', ' + df_1['Nearest Community'] + ', AK, USA'

In [22]:
df_1

,Campground,RV,Fee,# Sites,Picnic Sites,Toilets,Water,Trails,Fishing,Nearest Community,Location,Winter Open,Address
0,Bird Creek,Y,$20,27,Y,H,Y,Y,Y,Girdwood,Mi. 101.2 Seward Hwy,,"Bird Creek, Mi. 101.2 Seward Hwy, Girdwood, AK..."
1,Bird Creek Overflow,Y,$20,20,Y,H,H,H,Y,Girdwood,,,"Bird Creek Overflow, , Girdwood, AK, USA"
2,Eagle River*,,$25,57,Y,H,H,N,Y,Eagle River,Mi. 12.6 Glenn Hwy,,"Eagle River*, Mi. 12.6 Glenn Hwy, Eagle River,..."
3,Eklutna,,$15,58,H,H,H,Y,Y,Eklutna,Mi. 26.5 Glenn Hwy,,"Eklutna, Mi. 26.5 Glenn Hwy, Eklutna, AK, USA"
4,Big Delta SHP,Y,$20,25,Y,H,H,Y,N,Delta Junction,Mi. 274.5 Richardson Hwy,,"Big Delta SHP, Mi. 274.5 Richardson Hwy, Delta..."
5,Birch Lake SRS,Y,$15,19,Y,H,Y,N,Y,Fairbanks,Mi. 305.5 Richardson Hwy,,"Birch Lake SRS, Mi. 305.5 Richardson Hwy, Fair..."
6,Red Squirrel,Y,$15,5,H,H,Y,N,Y,Fairbanks,Mi. 43 Chena Hot Springs Rd,,"Red Squirrel, Mi. 43 Chena Hot Springs Rd, Fai..."
7,Rosehip,Y,$15,37,Y,H,Y,Y,Y,Fairbanks,Mi. 27 Chena Hot Springs Rd,,"Rosehip, Mi. 27 Chena Hot Springs Rd, Fairbank..."
8,Granite Tors,Y,$15,24,H,H,Y,Y,Y,Fairbanks,Mi. 39 Chena Hot Springs Rd,,"Granite Tors, Mi. 39 Chena Hot Springs Rd, Fai..."
9,Chena River SRS**,Y,$15/$30,61,H,H,H,Y,Y,Fairbanks,3530 Geraghty Ave,,"Chena River SRS**, 3530 Geraghty Ave, Fairbank..."


In [24]:
df = df_1.copy()
applied_df = df.apply(lambda row: geocode(row.Address), axis='columns', result_type='expand')
df = pd.concat([df, applied_df], axis='columns')

df = df.rename(columns={0:'lat', 1: 'lng'})
df

,Campground,RV,Fee,# Sites,Picnic Sites,Toilets,Water,Trails,Fishing,Nearest Community,Location,Winter Open,Address,lat,lng
0,Bird Creek,Y,$20,27,Y,H,Y,Y,Y,Girdwood,Mi. 101.2 Seward Hwy,,"Bird Creek, Mi. 101.2 Seward Hwy, Girdwood, AK...",60.933641,-149.156054
1,Bird Creek Overflow,Y,$20,20,Y,H,H,H,Y,Girdwood,,,"Bird Creek Overflow, , Girdwood, AK, USA",60.941040,-149.172700
2,Eagle River*,,$25,57,Y,H,H,N,Y,Eagle River,Mi. 12.6 Glenn Hwy,,"Eagle River*, Mi. 12.6 Glenn Hwy, Eagle River,...",61.316480,-149.574140
3,Eklutna,,$15,58,H,H,H,Y,Y,Eklutna,Mi. 26.5 Glenn Hwy,,"Eklutna, Mi. 26.5 Glenn Hwy, Eklutna, AK, USA",61.455170,-149.366020
4,Big Delta SHP,Y,$20,25,Y,H,H,Y,N,Delta Junction,Mi. 274.5 Richardson Hwy,,"Big Delta SHP, Mi. 274.5 Richardson Hwy, Delta...",63.793616,-145.750973
5,Birch Lake SRS,Y,$15,19,Y,H,Y,N,Y,Fairbanks,Mi. 305.5 Richardson Hwy,,"Birch Lake SRS, Mi. 305.5 Richardson Hwy, Fair...",64.820500,-147.700540
6,Red Squirrel,Y,$15,5,H,H,Y,N,Y,Fairbanks,Mi. 43 Chena Hot Springs Rd,,"Red Squirrel, Mi. 43 Chena Hot Springs Rd, Fai...",64.888235,-147.622911
7,Rosehip,Y,$15,37,Y,H,Y,Y,Y,Fairbanks,Mi. 27 Chena Hot Springs Rd,,"Rosehip, Mi. 27 Chena Hot Springs Rd, Fairbank...",64.888235,-147.622911
8,Granite Tors,Y,$15,24,H,H,Y,Y,Y,Fairbanks,Mi. 39 Chena Hot Springs Rd,,"Granite Tors, Mi. 39 Chena Hot Springs Rd, Fai...",64.888235,-147.622911
9,Chena River SRS**,Y,$15/$30,61,H,H,H,Y,Y,Fairbanks,3530 Geraghty Ave,,"Chena River SRS**, 3530 Geraghty Ave, Fairbank...",64.837757,-147.806355


In [25]:
df[pd.isnull(df.lat)]

,Campground,RV,Fee,# Sites,Picnic Sites,Toilets,Water,Trails,Fishing,Nearest Community,Location,Winter Open,Address,lat,lng


In [ ]:
anchorage = geocode('Anchorage, Alaska, USA')
map2 = folium.Map(location=(anchorage[0], anchorage[1]), zoom_start=5, control_scale=True)
# add marker one by one on the map
for idx, row in df.iterrows():
    if not pd.isnull(row['lat']) and not pd.isnull(row['lng']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            popup=row['Campground'],
        ).add_to(map2)

### Map of State of Alaska Campgrounds

In [27]:
map2